In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))
print("/kaggle/inputfruits/fruits-360_dataset/fruits-360/Training/")
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import tensorflow as tf
import zipfile
import pathlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random
import os
from skimage.transform import resize
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Sequential
import random

In [8]:
train_dir = "/kaggle/input/fruits/fruits-360_dataset/fruits-360/Training/"
test_dir = "/kaggle/input/fruits/fruits-360_dataset/fruits-360/Test/"

In [9]:
# Get class names
import pathlib
data_dir = pathlib.Path(train_dir)
class_names = np.array(sorted([item.name for item in data_dir.glob('*')]))
print(class_names,len(class_names))

In [10]:
# Get class names
import pathlib
data_dir = pathlib.Path(test_dir)
class_names = np.array(sorted([item.name for item in data_dir.glob('*')]))
print(class_names,len(class_names))

In [11]:
sel = random.sample(list(class_names),10)

In [12]:
# Visualize
def view_random_image(target_dir,target_class):
  target_folder = target_dir+"/"+target_class
  random_image = random.sample(os.listdir(target_folder),1) # This will be returned as a list, so we use [0] below
  img = mpimg.imread(target_folder+"/"+random_image[0])
  fig = plt.imshow(img)
  plt.title(target_class)
  # plt.axes('off')
  fig.axes.get_xaxis().set_visible(False)
  fig.axes.get_yaxis().set_visible(False)  
  print("Image shape:",img.shape)
  return img

In [8]:
img = view_random_image(train_dir,"Apple Golden 3")

In [13]:
train_datagen = ImageDataGenerator(rescale=1/255.)
test_datagen = ImageDataGenerator(rescale=1/255.)

In [14]:
train_data = train_datagen.flow_from_directory(directory=train_dir,
                                               target_size =(224,224),
                                               batch_size=256,
                                               class_mode="categorical",
                                              classes=sel)
test_data = test_datagen.flow_from_directory(directory=test_dir,
                                             target_size=(224,224),
                                             batch_size=256,
                                             class_mode="categorical",
                                            classes=sel)

In [15]:
len(train_data),len(test_data)

In [3]:
# Building the model
m1 = Sequential([
    Conv2D(filters=8,kernel_size=5,input_shape=(224,224,3),activation="relu"),
    MaxPool2D(),

    Conv2D(filters=16,kernel_size=4,activation="relu"),
    MaxPool2D(),

    Conv2D(filters=32,kernel_size=3,activation="relu"),
    tf.keras.layers.GlobalAveragePooling2D(),

    Flatten(),
    Dense(10,activation="softmax")
])
m1.compile(
    loss=tf.keras.losses.categorical_crossentropy,
    optimizer=Adam(learning_rate=0.01),
    metrics=['accuracy']
)

In [6]:
import visualkeras as vk

In [7]:
vk.layered_view(m1)

In [29]:
h1 = m1.fit(
    train_data,epochs=5,
    validation_data=test_data,
    steps_per_epoch=len(train_data),
    validation_steps=len(test_data)
)

In [30]:
pd.DataFrame(h1.history).plot()

### Model 2 will have the architecture given below
![](https://www.researchgate.net/profile/Jose-Naranjo-Torres/publication/341455419/figure/fig3/AS:892525732712453@1589806125572/Designed-CNN-architecture-for-fruit-classification.png)

In [16]:
# Let us try changing batch size and try with more change in model architecture
train_data = train_datagen.flow_from_directory(directory=train_dir,
                                               target_size =(224,224),
                                               batch_size=512,
                                               class_mode="categorical",
                                              classes=sel)
test_data = test_datagen.flow_from_directory(directory=test_dir,
                                             target_size=(224,224),
                                             batch_size=512,
                                             class_mode="categorical",
                                            classes=sel)
m2 = Sequential([
    Conv2D(filters=8,kernel_size=5,input_shape=(224,224,3),activation="relu"),
    tf.keras.layers.Normalization(),
    MaxPool2D(),

    Conv2D(filters=16,kernel_size=4,activation="relu"),
    tf.keras.layers.Normalization(),
    MaxPool2D(),

    Conv2D(filters=32,kernel_size=3,activation="relu"),
    tf.keras.layers.GlobalAveragePooling2D(),

    Flatten(),
    Dense(10,activation="softmax")
])
m2.compile(
    loss=tf.keras.losses.categorical_crossentropy,
    optimizer=Adam(learning_rate=0.01),
    metrics=['accuracy']
)

In [17]:
vk.layered_view(m2)

In [33]:
h2 = m2.fit(
    train_data,epochs=10,
    validation_data=test_data,
    steps_per_epoch=len(train_data),
    validation_steps=len(test_data)
)

In [34]:
# Finally the model has reached above 90 accuracy scores 😇😇😇
pd.DataFrame(h2.history).plot()

In [ ]:
# Let us try to make VGG16 architecture with change in output layer

![](https://miro.medium.com/max/1400/0*0M8CobXpNwFDCmOQ)

In [19]:
vgg16 = Sequential([
    Conv2D(64,kernel_size=3,input_shape=(224,224,3),activation="relu",padding="same"),
    Conv2D(64,kernel_size=3,padding="same",activation="relu"),
    MaxPool2D(pool_size=2,strides=2),
    
    Conv2D(128,kernel_size=3,padding="same",activation="relu"),
    Conv2D(128,kernel_size=3,padding="same",activation="relu"),
    MaxPool2D(pool_size=2,strides=2),
    
    Conv2D(256,kernel_size=3,padding="same",activation="relu"),
    Conv2D(256,kernel_size=3,padding="same",activation="relu"),
    Conv2D(256,kernel_size=3,padding="same",activation="relu"),
    MaxPool2D(pool_size=2,strides=2),
    
    Conv2D(512,kernel_size=3,padding="same",activation="relu"),
    Conv2D(512,kernel_size=3,padding="same",activation="relu"),
    Conv2D(512,kernel_size=3,padding="same",activation="relu"),
    MaxPool2D(pool_size=2,strides=2),
    
    Conv2D(512,kernel_size=3,padding="same",activation="relu"),
    Conv2D(512,kernel_size=3,padding="same",activation="relu"),
    Conv2D(512,kernel_size=3,padding="same",activation="relu"),
    MaxPool2D(pool_size=2,strides=2),
    
    Flatten(),
    Dense(256,activation="relu",name="fc1"),
    Dense(128,activation="relu",name="fc2"),
    Dense(10,activation="softmax",name="output")
])
vgg16.compile(
    loss=tf.keras.losses.categorical_crossentropy,
    optimizer=Adam(learning_rate=0.001),
    metrics=['accuracy']
)

In [21]:
vk.layered_view(vgg16,legend=True)

In [25]:
# h_vgg16 = vgg16.fit(
#     train_data,epochs=5,
#     validation_data=test_data,
#     steps_per_epoch=len(train_data),
#     validation_steps=len(test_data)
# )
# The notebook crashes if we try to train VGG16. I will have to use transfer learning for this.

In [28]:
# Let us try a model similar to VGG but with less filters
vgg16_changed = Sequential([
    Conv2D(4,kernel_size=3,input_shape=(224,224,3),activation="relu",padding="same"),
    Conv2D(4,kernel_size=3,padding="same",activation="relu"),
    MaxPool2D(pool_size=2,strides=2),
    
    Conv2D(8,kernel_size=3,padding="same",activation="relu"),
    Conv2D(8,kernel_size=3,padding="same",activation="relu"),
    MaxPool2D(pool_size=2,strides=2),
    
    Conv2D(16,kernel_size=3,padding="same",activation="relu"),
    Conv2D(16,kernel_size=3,padding="same",activation="relu"),
    MaxPool2D(pool_size=2,strides=2),
    
    Conv2D(32,kernel_size=3,padding="same",activation="relu"),
    Conv2D(32,kernel_size=3,padding="same",activation="relu"),
    MaxPool2D(pool_size=2,strides=2),
    
    Conv2D(16,kernel_size=3,padding="same",activation="relu"),
    Conv2D(16,kernel_size=3,padding="same",activation="relu"),
    MaxPool2D(pool_size=2,strides=2),
    
    Flatten(),
    Dense(10,activation="softmax",name="output")
])
vgg16_changed.compile(
    loss=tf.keras.losses.categorical_crossentropy,
    optimizer=Adam(learning_rate=0.001),
    metrics=['accuracy']
)

In [29]:
vk.layered_view(vgg16_changed,legend=True)

In [30]:
vgg16_changed = vgg16_changed.fit(
    train_data,epochs=10,
    validation_data=test_data,
    steps_per_epoch=len(train_data),
    validation_steps=len(test_data)
)

In [ ]:
# That's amazing, this changed VGG16 worked even better than my previous models

In [31]:
pd.DataFrame(vgg16_changed.history).plot()
# That's great !!